In [3]:
import pandas as pd
import numpy as np
import pickle
from mood.base.sequence import Sequence


In [5]:
native = Sequence(sequence="HNPVVMVHGMGGASYNFASIKSYLV", chain="A", index=0, parent=False, child=False, active=True),

sequences = {
    "A":
    {
        "HNPVVMVHGMGGASYNFASIKSYLV": Sequence(sequence="HNPVVMVHGMGGASYNFASIKSYLV", chain="A", index=0, parent=False, child=False, active=True), 
        "HNPVVMVHGMGGASYNFASIKSLLV": Sequence(sequence="HNPVVMVHGMGGASYNFASIKSLLV", chain="A", index=1, parent=False, child=False, active=True, native=native),
        "HNPVVMVHGMGGASYNFASIKSSLV": Sequence(sequence="HNPVVMVHGMGGASYNFASIKSSLV", chain="A", index=2, parent=False, child=False, active=True, native=native),
        "HNPVVMVHGMGGASYNFASIKSVLV": Sequence(sequence="HNPVVMVHGMGGASYNFASIKSVLV", chain="A", index=3, parent=False, child=False, active=True, native=native),
        "HNPVVMVHGMGGASYNFASIKSKLV": Sequence(sequence="HNPVVMVHGMGGASYNFASIKSKLV", chain="A", index=4, parent=False, child=False, active=True, native=native),
    }
}
with open("sequences.pkl", "wb") as f:
    pickle.dump(sequences, f)

In [3]:
seq_index = [0,1,2,3,4]
sequence = ["HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV"]
iteration = [1,2,3,4,5]
metric1 = [-2, -6, -1, -1, -5]
metric2 = [-1, -6, -1, -3, -5]
rank = [1, 2, 3, 4, 5]
df = pd.DataFrame({'seq_index': seq_index, 'Sequence': sequence, 'iteration': iteration, 'metric1': metric1, 'metric2': metric2, 'Rank': rank})
dataframes = {"A": df}
with open("data_frame.pkl", "wb") as f:
    pickle.dump(dataframes, f)

In [1]:

SEQ1 = Sequence(
    sequence="ATG",
    chain="A",
    index=1,
    active=True,
    parent=None,
    child=None,
    native=None,
)
SEQ2 = Sequence(
    sequence="TGG",
    chain="A",
    index=2,
    active=True,
    parent=None,
    child=None,
    native=SEQ1,
)
SEQ3 = Sequence(
    sequence="GAT",
    chain="A",
    index=3,
    active=True,
    parent=None,
    child=None,
    native=SEQ1,
)
SEQUENCES_C = {
    "A": {
        SEQ1.sequence: SEQ1,
        SEQ2.sequence: SEQ2,
        SEQ3.sequence: SEQ3,
    }
}